## Upload file

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Preprocessing1.csv to Preprocessing1.csv


## Read CSV file to pandas

In [3]:
import pandas as pd
data = pd.read_csv('Preprocessing1.csv')
data

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month,WeekDay,Day
0,Jet Airways,Delhi,Banglore,DEL → BOM → COK,20:00,04:25 10 Jun,26h 35m,1 stop,In-flight meal not included,14924,6,Thursday,6.0
1,Jet Airways,Delhi,Cochin,DEL → BOM → COK,16:00,19:00 10 Jun,27h,1 stop,In-flight meal not included,10577,6,Sunday,9.0
2,Jet Airways,Mumbai,Hyderabad,BOM → HYD,19:35,21:05,1h 30m,non-stop,No info,5678,3,Friday,15.0
3,Multiple carriers,Delhi,Banglore,DEL → BOM → COK,18:55,01:30 16 Jun,15h 10m,1 stop,In-flight meal not included,7408,5,Monday,6.0
4,Air India,Delhi,Cochin,DEL → COK,17:10,17:55,8h 20m,non-stop,No info,6724,6,Monday,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9445,Air India,Banglore,Cochin,BLR → MAA → DEL,19:30,19:50,8h 55m,1 stop,In-flight meal not included,5932,6,Wednesday,12.0
9446,IndiGo,Delhi,Cochin,DEL → BOM → COK,17:00,21:00,8h 35m,1 stop,No info,6838,3,Sunday,3.0
9447,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,07:05,19:45,15h 15m,1 stop,No info,12615,3,Saturday,9.0
9448,Jet Airways,Kolkata,Banglore,CCU → BOM → BLR,08:20,19:50,12h 5m,1 stop,No info,10844,5,Saturday,18.0


## Average of flight ticket price

In [6]:
data['Price'].mean()

9027.895555555555

## Month - Highest Number of Flights

In [17]:
group_by_month_count = data.groupby('Month')['Airline'].count().reset_index()
print(group_by_month_count)

   Month  Airline
0      3     2388
1      4      926
2      5     3092
3      6     3044


## Average price

In [35]:
group_by_weekday = data.groupby('WeekDay')['Airline'].count().reset_index()
group_by_weekday

,WeekDay,Airline
0,Friday,834
1,Monday,1597
2,Saturday,1546
3,Sunday,1253
4,Thursday,1590
5,Tuesday,737
6,Wednesday,1893


In [52]:
weekday_criteria = lambda a: a.WeekDay.isin(['Monday','Tuesday','Wednesday','Thursday', 'Friday'])
weekend_critera = lambda a: a.WeekDay.isin(['Saturday','Sunday'])

In [57]:
weekday = group_by_weekday[weekday_criteria]['Airline'].mean()
weekend = group_by_weekday[weekend_critera]['Airline'].mean()
print("weekday=", weekday)
print("weekend=", weekend)

weekday= 1330.2
weekend= 1399.5


## Replace 'No Info' with 'No info' in Addition information column

In [40]:
# data.columns
criteria = data['Additional_Info'] == "No Info"
data.loc[criteria, ['Additional_Info']] = "No info"

In [51]:
criteria_1 = data.Airline == "IndiGo"
criteria_2 = data.Additional_Info=="No info"
data.loc[criteria_1 & criteria_2].shape[0]

1650

## Convert Duration to Seconds

In [57]:
value_map = lambda a : int(a["Duration"].split('h')[0]) * 60 * 60 + (a["Duration"].split('h')[1].replace('m','')) * 60

In [75]:
def map_duration(duration):
  parts = duration.split()
  hours = 0
  minutes = 0
  for part in parts:
      if 'h' in part:
          hours = int(part.replace('h', ''))
      if 'm' in part:
          minutes = int(part.replace('m', ''))
  return hours * 3600 + minutes * 60

In [77]:
data["Duration in Sec"] = data["Duration"].apply(map_duration)

38957.93650793651

In [79]:
data["Duration in Sec"].mean()

38957.93650793651

## Number of Flights started in morning and departed in Evening

In [87]:
def transform_time_category(time):
  hr = int(time.split(sep=":")[0])
  if 5 <= hr < 12:
    return "Morning"
  elif 12 <= hr < 17:
    return "Afternoon"
  elif 17 <= hr < 20:
    return "Evening"
  else:
    return "Night"

In [88]:
data["Dep_Time Category"] = data["Dep_Time"].apply(transform_time_category)

In [89]:
data["Dep_Arrival_TimeTime Category"] = data["Arrival_Time"].apply(transform_time_category)

In [91]:
data[(data['Dep_Arrival_TimeTime Category']=='Evening') & (data['Dep_Time Category']=='Morning')].shape[0]

922

## Encode WeekDay

In [92]:
def encode_weekday(weekday):
  if (weekday in ['Sunday' , 'Saturday']):
    return 1
  else:
    return 0

In [93]:
data['Weekend'] = data['WeekDay'].apply(encode_weekday)

In [94]:
data[data['Weekend']==1].shape[0]

2799

In [96]:
data[data['Weekend']==0].shape[0]

6651